# Subscriptions API: Planet SDK Quickstart


Getting started with Planet SDK and the Subscriptions API.

The Subscriptions API is Planet's next-gen data delivery API; subscribe to continuous delivery of imagery and metadata collections which meet your item filter criteria with a single API call. Subscriptions must be delivered or hosted on cloud storage and cannot be delivered to your local machine like deliveries from the Data or Orders API's.


This tutorial will take a look at the functionality of the Subscription API with the Python SDK and how you might be able to incorporate it into your own workflows.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

For Subscriptions, local delivery is not supported and subscriptions must be delivered to a cloud bucket or hosting service. For details on delivery schema for the supported cloud delivery platforms, see the [Subscriptions API Delivery Documentation](https://developers.planet.com/docs/subscriptions/delivery/).

Supported Cloud platforms include:
- Google Cloud Storage (used in this tutorial)
- Google Earth Engine
- Amazon S3 and other S3-compatible cloud storage providers
- Oracle Cloud Storage
- Microsoft Azure
<br>
Note that subscriptions can also use the data hosting feature on Sentinel Hub/Planet Insights Platform, and 'hosting' is used instead of 'delivery' in API calls.
<br> <br>

Regardless of the platform you're using, ensure the credentials you provide have **both write and delete access** to your storage location.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Subscriptions API Documentation](https://docs.planet.com/develop/apis/subscriptions/)

_____

## Setup

In order to interact with the Planet API using the Python client, we need to authenticate our Planet account credentials and import the necessary packages.

In [1]:
import os
from datetime import datetime
from planet import Auth, Session, Planet, subscription_request

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [2]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=True)
if not auth.is_initialized():
    auth.user_login(allow_open_browser=False, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

_____

## Listing a Subscription

First, let's try listing any existing subscriptions we already have in our Planet accounts. 
<br>
If you've never made a subscription before, this will show up as an empty list. <br> <br> We'll try this again below after we've created our first subscription.

In [ ]:
existing_subscriptions = list(pl.subscriptions.list_subscriptions(status=['running', 'pending', 'completed'], limit=5))

In [37]:
existing_subscriptions

[]

_______

## Creating a Subscription

**WARNING: Creating a Request below will consume quota! Be sure about the filter params so you can create a useful subscription. The variables below are meant as placeholders.**

In order to create a subscription using the Python Client, we must provide a subscription request. This will include a `name`, `source` block, along with a `delivery` or a `hosting` block with a specified cloud storage location or cloud hosting platform, respectively. Subscriptions also support select tools. We will accomplish this all using `build_request()`.

#### Cloud Delivery Parameters:

Your delivery or hosting parameters may be specified here. Below is a template for delivery to a Google Cloud Storage bucket. Note that you will need a service account with create, delete, and get permissions. It is recommended to store your GCS credentials as an environment variable. See the [Google Cloud documentation](https://cloud.google.com/iam/docs/keys-create-delete#iam-service-account-keys-create-console) on how to create a service account key with the appropriate permissions. More information about GCS hosting is available [here](https://docs.planet.com/develop/apis/subscriptions/delivery/#google-cloud-storage). 

In [ ]:
# Google Cloud Storage placeholder
gcs_key = os.getenv('GCS_CREDENTIALS')
bucket_name = 'your_bucket_name'
# optional path_prefix
path_prefix = ''
bucket_delivery = subscription_request.google_cloud_storage(
    credentials=gcs_key, bucket=bucket_name, path_prefix=path_prefix)

#### Alternative to cloud delivery: Sentinel Hub Hosting

You may wish to host your subscription data with a Cloud hosting service like a Sentinel Hub Collection or Google Earth Engine. The `hosting` arg should be used *instead* of the `delivery` arg below in `create_subscription()`. Cloud hosting documentation for subscriptions can be found [here](https://docs.planet.com/develop/apis/subscriptions/delivery/#hosting).

In [ ]:
# Sentinel Hub collection hosting
collection_id = 'collection_id'
collection_hosting = subscription_request.sentinel_hub(collection_id=collection_id)

In [ ]:
# GEE collection hosting
project = 'project_name'
collection = 'collection_name'
credentials = 'gee_credentials'
collection_hosting = subscription_request.google_earth_engine(project, collection, credentials)

In [ ]:
collection_hosting

____

#### Build your Subscription
For this example, our `source` block will be created from `subscription_request.catalog_source()` and will contain item_types, asset_types, geometry, start_time, end_time. 


Alternatively you may need `subscription_request.subscription_source` for Planetery variables or Analysis Ready PlanetScope. More info for subscription_source can be found [here](https://docs.planet.com/develop/apis/subscriptions/sources/#planetary-variable-and-analysis-ready-source-types).

The `rrule` attribute leverages [iCalendar recurrence rules](https://icalendar.org/iCalendar-RFC-5545/3-8-5-3-recurrence-rule.html). This can be used to create subscriptions that deliver items for recurring periods within the total coverage time. In this example, we are getting monthly data between March through October, from 2022 to 2025. More information on how to use this attribute can be found [here](https://developers.planet.com/docs/subscriptions/source/#rrules-recurrence-rules).

Feel free to use the below request format as a template to input the parameters you need for your own subscription.

In [9]:
# Example catalog parameters
item_types = ["PSScene"]
asset_types = ["ortho_visual"]

geometry = {
        "coordinates": [[
           [-122.33026701728114, 37.75207301385534],
           [-122.20961421268103, 37.75207301385534],
           [-122.20961421268103, 37.84571423146804],
           [-122.33026701728114, 37.84571423146804],
           [-122.33026701728114, 37.75207301385534]]],
        "type": "Polygon"
      }

start_time = datetime.fromisoformat("2025-09-01T00:00:00Z")
end_time = datetime.fromisoformat("2025-11-01T00:00:00Z")
rrule = "FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10"

source = subscription_request.catalog_source(
    item_types=item_types, asset_types=asset_types, geometry=geometry,
    start_time=start_time, end_time=end_time, rrule=rrule)

source

{'parameters': {'item_types': ['PSScene'],
  'asset_types': ['ortho_visual'],
  'geometry': {'coordinates': [[[-122.33026701728114, 37.75207301385534],
     [-122.20961421268103, 37.75207301385534],
     [-122.20961421268103, 37.84571423146804],
     [-122.33026701728114, 37.84571423146804],
     [-122.33026701728114, 37.75207301385534]]],
   'type': 'Polygon'},
  'start_time': '2025-09-01T00:00:00+00:00',
  'end_time': '2025-11-01T00:00:00+00:00',
  'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}}

In [ ]:
sub_built = subscription_request.build_request(
    name="recurring_sub_test_api_quickstart", source=source,
    delivery=bucket_delivery)

In [14]:
sub_built

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'geometry': {'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]],
    'type': 'Polygon'},
   'start_time': '2025-09-01T00:00:00+00:00',
   'end_time': '2025-11-01T00:00:00+00:00',
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'test_bucket', 'credentials': '<redacted>'}}}

#### Create your subscription:

In [12]:
sub_created = pl.subscriptions.create_subscription(sub_built)

In [15]:
sub_created

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'type': 'catalog',
  'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'publishing_stages': ['standard', 'finalized'],
   'time_range_type': 'acquired',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]]},
   'start_time': '2025-09-01T00:00:00Z',
   'end_time': '2025-11-01T00:00:00Z',
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'simon-test-bucket-pl',
   'credentials': '<REDACTED>'}},
 'created': '2025-10-01T22:29:29.513542Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/33867e00-3f40-4116-88c9-1a7215255324',
  'results': 'https://api.planet.com/subscriptions/v1/3

___

## Listing a Subscription

Now that created a subscription, let's try listing our 5 most recent subscriptions associated with our Planet account. We should see the subscription we just created above as part of this.

You can filter the subscriptions listed by status- `preparing`, `running`, `pending`, `completed`, or `canceled`.

For this example, we will only list the subscriptions we have that have not been canceled. We will save the ID of the most recent subscription.

In [16]:
sub_list = list(pl.subscriptions.list_subscriptions(status=['preparing', 'running', 'pending', 'completed'], limit=5))
sub_to_check = sub_list[0]

In [18]:
sub_to_check

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'type': 'catalog',
  'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'publishing_stages': ['standard', 'finalized'],
   'time_range_type': 'acquired',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]]},
   'start_time': '2025-09-01T00:00:00Z',
   'end_time': '2025-11-01T00:00:00Z',
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'simon-test-bucket-pl',
   'credentials': '<REDACTED>'}},
 'created': '2025-10-01T22:29:29.513542Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/33867e00-3f40-4116-88c9-1a7215255324',
  'results': 'https://api.planet.com/subscriptions/v1/3

_____

## Get a Subscription

We can use our subscription ID for a given subscription to retrieve the corresponding subscription's description. Let's see this in action using the subscription ID we saved above.

In [19]:
sub_id = sub_to_check['id']
sub_desc = pl.subscriptions.get_subscription(sub_id)

In [20]:
sub_desc

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'type': 'catalog',
  'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'publishing_stages': ['standard', 'finalized'],
   'time_range_type': 'acquired',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]]},
   'start_time': '2025-09-01T00:00:00Z',
   'end_time': '2025-11-01T00:00:00Z',
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'simon-test-bucket-pl',
   'credentials': '<REDACTED>'}},
 'created': '2025-10-01T22:29:29.513542Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/33867e00-3f40-4116-88c9-1a7215255324',
  'results': 'https://api.planet.com/subscriptions/v1/3

___

## Update a Subscription

What if we see something about our subscription that we want to change? We can update a subscription by specifying the subscription ID whose attributes we want to alter, along with a new request with the updated parameters. 

**Note some restrictions apply after a subscription transitions to `running`:**
- Changes to the `item_types` fields are not allowed.
- Changes to `start_time` is not allowed if the `start_time` has already happened.
    - The edit will only apply to future item publications and deliveries and no items will be redelivered.
- Changes to `tools` are not allowed.
    - If tools was left as `None` when you created your subscription, the default tools need be to set when you update the subscription.
- Backfill subscriptions cannot be edited.
    - If you need to reorder archive items based on an updated filter or tool specifications, you can search for and order archive items with the Data API and Orders API.

In [21]:
# Let's look at our original request once more:
sub_desc

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'type': 'catalog',
  'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'publishing_stages': ['standard', 'finalized'],
   'time_range_type': 'acquired',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]]},
   'start_time': '2025-09-01T00:00:00Z',
   'end_time': '2025-11-01T00:00:00Z',
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'simon-test-bucket-pl',
   'credentials': '<REDACTED>'}},
 'created': '2025-10-01T22:29:29.513542Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/33867e00-3f40-4116-88c9-1a7215255324',
  'results': 'https://api.planet.com/subscriptions/v1/3

In [ ]:
# We will reuse our 'id', 'name' from the description above.
# We will use the same delivery variable from before.
# If you used any tools in your original subscription, make sure to reuse them.

sub_id = sub_desc['id']
sub_name = sub_desc['name']

Let's change our end time January 1st, change our end year to 2027, and only look at images every other month.

In [ ]:
# We will reuse all variables set above, changing only end_time and rrule:
end_time_update = datetime.fromisoformat("2027-01-01T00:00:00Z")
rrule_update = "FREQ=MONTHLY;BYMONTH=2,4,6,8,10,12"

source_update = subscription_request.catalog_source(
    item_types=item_types, asset_types=asset_types, geometry=geometry,
    start_time=start_time, end_time=end_time_update, rrule=rrule_update)

sub_update = subscription_request.build_request(
    name = sub_name, source = source_update, delivery=bucket_delivery)

In [24]:
sub_update

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'geometry': {'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]],
    'type': 'Polygon'},
   'start_time': '2025-09-01T00:00:00+00:00',
   'end_time': '2027-01-01T00:00:00+00:00',
   'rrule': 'FREQ=MONTHLY;BYMONTH=2,4,6,8,10,12'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'test_bucket', 'credentials': '<redacted>'}}}

Now, we can update our subscription with this new request:

In [28]:
updated_sub = pl.subscriptions.update_subscription(subscription_id=sub_id, request=sub_update)

Check out the new parameters of our subscription below. Notice that the frequency and end dates of the subscription have changed! Also note that this is still the same subscription ID as we were working with above.

In [29]:
updated_sub

{'name': 'recurring_sub_test_api_quickstart',
 'source': {'type': 'catalog',
  'parameters': {'item_types': ['PSScene'],
   'asset_types': ['ortho_visual'],
   'publishing_stages': ['standard', 'finalized'],
   'time_range_type': 'acquired',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.33026701728114, 37.75207301385534],
      [-122.20961421268103, 37.75207301385534],
      [-122.20961421268103, 37.84571423146804],
      [-122.33026701728114, 37.84571423146804],
      [-122.33026701728114, 37.75207301385534]]]},
   'start_time': '2025-09-01T00:00:00Z',
   'end_time': '2027-01-01T00:00:00Z',
   'rrule': 'FREQ=MONTHLY;BYMONTH=3,4,5,6,7,8,9,10'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'simon-test-bucket-pl',
   'credentials': '<REDACTED>'}},
 'created': '2025-10-01T22:29:29.513542Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/33867e00-3f40-4116-88c9-1a7215255324',
  'results': 'https://api.planet.com/subscriptions/v1/3

____

## Get Subscription Results

Now that we've made sure our subscription description matches what we want, we can see our subscription results using the `get_results` method. This step will only work for subscriptions that are already began running. 
If you get an empty list, it may mean that your subscription is still pending, especially if the start date of the subscription has not arrived yet.
<br> <br>
Check out the results description below. Note that we can see the both when we created the subscription and when we updated it, as well as the status. 
<br>


In [30]:
results_desc = pl.subscriptions.get_results(sub_id)
results_list = [result for result in results_desc]

In [31]:
results_list

[{'id': 'daef022a-d4ce-4c15-b905-28ad59e316e7',
  'status': 'success',
  'properties': {'item_id': '20250927_192504_20_253a',
   'item_types': ['PSScene']},
  'created': '2025-10-01T22:29:33.483662Z',
  'updated': '2025-10-01T22:31:05.351179Z',
  'completed': '2025-10-01T22:31:05.351179Z',
  'errors': {},
  'outputs': ['33867e00-3f40-4116-88c9-1a7215255324/20250927_192504_20_253a/20250927_192504_20_253a_metadata.json',
   '33867e00-3f40-4116-88c9-1a7215255324/20250927_192504_20_253a/20250927_192504_20_253a_3B_Visual.tif']},
 {'id': '7e6edaba-592d-427d-8aaf-654a1da721f6',
  'status': 'success',
  'properties': {'item_id': '20250923_192445_87_24b8',
   'item_types': ['PSScene']},
  'created': '2025-10-01T22:29:33.441197Z',
  'updated': '2025-10-01T22:39:07.97061Z',
  'completed': '2025-10-01T22:39:07.97061Z',
  'errors': {},
  'outputs': ['33867e00-3f40-4116-88c9-1a7215255324/20250923_192445_87_24b8/20250923_192445_87_24b8_metadata.json',
   '33867e00-3f40-4116-88c9-1a7215255324/20250923

_____

## Cancel A Subscription

We can cancel a subscription by using the `cancel_subscription` method and providing the subscription_id corresponding to the subscription we want to cancel.

In [32]:
# There is no return for this
pl.subscriptions.cancel_subscription(sub_id)

We can confirm this subscription is canceled by getting a description of this subscription using this ID. We should its status reported as 'canceled'.

In [33]:
cancelled_sub = pl.subscriptions.get_subscription(sub_id)

In [34]:
cancelled_sub['status']

'cancelled'

## Congratulations!

You now have all the tools you need to create and edit your own subscriptions using the Subscriptions API in the Python SDK. For more information on the Subscriptions API, please check out our [Subscriptions API  Documentation](https://developers.planet.com/docs/subscriptions/)